In [1]:
# !pip -q install git+https://github.com/eaedk/testing-zindi-package.git
# from zindi.user import Zindian
# USERNAME = "adetoromichael346@gmail.com" #@param {type : "string"}
# user = Zindian(username = USERNAME)
# user.select_a_challenge(reward = 'all', kind = 'hackathon', active = 'true')
# user.download_dataset(destination = "dataset")

In [2]:
# !pip install catboost optuna # installing catboost and optuna libraries

In [3]:
# dependencies
import re
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import gc
import string
import datetime
import nltk
nltk.download('averaged_perceptron_tagger');

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import FeatureHasher
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

from sklearn.ensemble import RandomForestClassifier
from catboost import Pool, CatBoostClassifier
from sklearn.cluster import KMeans
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
# import optuna
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.metrics import log_loss, accuracy_score, classification_report

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\MIKE\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
#Reduce Memory Usage
def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtype
    
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    return df

In [5]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sub = pd.read_csv('SampleSubmission.csv')
random_seed = 101 # random seed for all computations

In [6]:
# display(train.head(), test.head())

In [7]:
train.shape, test.shape

((57974, 8), (22625, 8))

In [8]:
# train['DATE'] = pd.to_datetime(train['DATE'], format='%Y-%m-%d')
# test['DATE'] = pd.to_datetime(test['DATE'], format='%Y-%m-%d')

In [9]:
train = train.sort_values('DATE').reset_index(drop=True) 
test = test.sort_values('DATE').reset_index(drop=True)

# for dataset in (train, test):
#     dataset['Day'] = pd.to_datetime(dataset['DATE']).dt.day
#     # dataset['Day'] = dataset.DATE.dt.day
#     # dataset['Month'] = dataset.DATE.dt.month
#     # dataset['Year'] = dataset.DATE.dt.year
#     # dataset.set_index('DATE', inplace = True)

In [10]:
# Hypotheses
# Transaction details will have a huge impact in determining which category the transaction belong to.
# Date might also not has a great impact on the transaction category, so it will be dropped
# Due to behaviour on customers, account no. which symbolizes a customer might has a little impact in determining the category of a transaction

In [11]:
# Train['TRANSACTION DETAILS'].value_counts()

In [12]:
# display(Train.isna().sum(), Test.isna().sum())
# Train['TRANSACTION DETAILS'].unique()
# I will drop cheque no. due to presence of huge missing values

In [13]:
# for name, group in train.groupby(train.Category):
#     print(name)
#     print(group)
#     print('\n')

In [14]:
# cols = Train.columns
# def card(cols):
#   for col in cols:
#     print(f'{col}: {Train[col].nunique()}')
# card(cols)
# print('len of data: ', Train.shape[0])
# # the balance amount might not really matter in determining the category and it also has high cardinality, so it will be dropped

In [15]:
# #Target Histogram
# plt.figure(figsize=(20, 15))
# sns.set_style("white") 
# sns.countplot( y='Category', data=Train, palette="Set1");

In [16]:
# Train.describe(include = 'all').T

In [17]:
# #Numerical Features Histograms (Train)
# num_feats = list(Train.select_dtypes(include=['int64', 'float64', 'int32']).columns)
# Train[num_feats].hist(figsize=(20,15));

In [18]:
# #Target VS USER_GENDER 
# sns.countplot( y='Account_NO', data=Train, palette="Set1");

In [19]:
# train.iloc[np.where(train['TRANSACTION DETAILS'].isnull() == True)]

In [20]:
train = train.dropna(how='all')
test = test.dropna(how='all')

In [21]:
train.dropna(subset = ['Category'], inplace = True)
train.shape

(56203, 8)

In [22]:
ID = test.ID
test.drop('ID', inplace = True, axis = 1)

In [23]:
# display(train.isna().sum(), test.isna().sum())

In [24]:
train['TRANSACTION DETAILS'].fillna('NEFT/FDRL401249531/INDIAFORENSIC', inplace = True)

In [25]:
train.columns

Index(['DATE', 'TRANSACTION DETAILS', 'Account_NO', 'CHQ.NO.',
       'WITHDRAWAL AMT', 'DEPOSIT AMT', 'BALANCE AMT', 'Category'],
      dtype='object')

In [28]:
data = pd.concat([train, test], axis = 0)
# def Agg_1(Feature):
#     for dataset in (train, test):
#         for key in ['WITHDRAWAL AMT', 'DEPOSIT AMT']:
#             dataset[f'{Feature}_{key}_Min'] = dataset[key].map(dict(data.groupby(key)[Feature].min()))
#             dataset[f'{Feature}_{key}_Max'] = dataset[key].map(dict(data.groupby(key)[Feature].max()))
#             dataset[f'{Feature}_{key}_Mean'] = dataset[key].map(dict(data.groupby(key)[Feature].mean()))
            
# Agg_1('Account_NO')

In [29]:
def Agg_2(Feature):
    for dataset in (train, test):
        for key in ['TRANSACTION DETAILS']:
            dataset[f'{Feature}_{key}_Mode'] = dataset[key].map(dict(data.groupby(key)[Feature].apply(lambda x: x.value_counts().keys()[0])))
            dataset[f'{Feature}_{key}_Count'] = dataset[key].map(dict(data.groupby(key)[Feature].count()))
            dataset[f'{Feature}_{key}_NUnique'] = dataset[key].map(dict(data.groupby(key)[Feature].nunique()))
            
Agg_2('Account_NO')

In [30]:
train['isGreater'] = np.where(train['DEPOSIT AMT'] > train['WITHDRAWAL AMT'], 1, 0)
test['isGreater'] = np.where(test['DEPOSIT AMT'] > test['WITHDRAWAL AMT'], 1, 0)

In [31]:
train.columns

Index(['DATE', 'TRANSACTION DETAILS', 'Account_NO', 'CHQ.NO.',
       'WITHDRAWAL AMT', 'DEPOSIT AMT', 'BALANCE AMT', 'Category',
       'Account_NO_TRANSACTION DETAILS_Mode',
       'Account_NO_TRANSACTION DETAILS_Count',
       'Account_NO_TRANSACTION DETAILS_NUnique', 'isGreater'],
      dtype='object')

In [32]:
for dataset in (train, test):
    dataset.drop(['CHQ.NO.', 'WITHDRAWAL AMT', 'DEPOSIT AMT', 'BALANCE AMT', 'DATE'], axis = 1, inplace = True)
# , 'Account_NO', 'WITHDRAWAL AMT', 'DEPOSIT AMT', 'BALANCE AMT'

In [33]:
gc.collect()

766

In [34]:
# reduce_memory_usage(train), reduce_memory_usage(test)

In [35]:
# data = pd.concat([train, test], sort = False).reset_index(drop = True)
# data.shape

In [36]:
train['TRANSACTION DETAILS'] = train['TRANSACTION DETAILS'].str.replace('/', '')
test['TRANSACTION DETAILS'] = test['TRANSACTION DETAILS'].str.replace('/', '')

In [37]:
train.shape, test.shape

((56203, 7), (22625, 6))

In [60]:
# tfidf = TfidfVectorizer(min_df = 4, max_features = 500, analyzer = 'char', stop_words = 'english', ngram_range = (3, 5), dtype = np.float32)
# tfidf.fit(train['TRANSACTION DETAILS'])
# train_feat = tfidf.transform(train['TRANSACTION DETAILS'])
# test_feat = tfidf.transform(test['TRANSACTION DETAILS'])
# vec_train = pd.DataFrame(train_feat.todense(), columns = tfidf.get_feature_names())
# vec_test = pd.DataFrame(test_feat.todense(), columns = tfidf.get_feature_names())

In [38]:
# vec_train.columns.to_list()

In [61]:
# train = pd.concat([train.reset_index(), vec_train], axis = 1)
# test = pd.concat([test.reset_index(), vec_test], axis = 1)

In [62]:
# test.isna().sum()

In [63]:
def tag_part_of_speech_noun(text):
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    pos_list = nltk.pos_tag(text_splited)
    noun_count = len([w for w in pos_list if w[1] in ('NN','NNP','NNPS','NNS')])
    return noun_count

def tag_part_of_speech_adj(text):
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    pos_list = nltk.pos_tag(text_splited)
    adjective_count = len([w for w in pos_list if w[1] in ('JJ','JJR','JJS')])
    return adjective_count

def tag_part_of_speech_verb(text):
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    pos_list = nltk.pos_tag(text_splited)
    verb_count = len([w for w in pos_list if w[1] in ('VB','VBD','VBG','VBN','VBP','VBZ')])
    return verb_count

In [64]:
for df in ([train, test]):
    df['TRANSACTION_DETAILS_Length'] = df['TRANSACTION DETAILS'].apply(lambda x : len(x))
    df['Num_Words'] = df['TRANSACTION DETAILS'].apply(lambda comment: len(comment.split()))
    df['Num_Unique_Words'] = df['TRANSACTION DETAILS'].apply(lambda comment: len(set(w for w in comment.split())))
    df['Words_VS_Unique'] = df['Num_Unique_Words'] / df['Num_Words']
    df['num_punctuation'] = df['TRANSACTION DETAILS'].apply(lambda comment: sum(comment.count(w) for w in '\'.,;:'))
    df['nouns'] = df['TRANSACTION DETAILS'].apply(lambda comment: tag_part_of_speech_noun(comment))
    df['adjectives'] = df['TRANSACTION DETAILS'].apply(lambda comment: tag_part_of_speech_adj(comment))
    df['verbs'] = df['TRANSACTION DETAILS'].apply(lambda comment: tag_part_of_speech_verb(comment))
    df['nouns_vs_length'] = df['nouns'] / df['TRANSACTION_DETAILS_Length']
    df['adjectives_vs_length'] = df['adjectives'] / df['TRANSACTION_DETAILS_Length']
    df['verbs_vs_length'] = df['verbs'] / df['TRANSACTION_DETAILS_Length']
    df['nouns_vs_words'] = df['nouns'] / df['Num_Words']
    df['adjectives_vs_words'] = df['adjectives'] / df['Num_Words']
    df['verbs_vs_words'] = df['verbs'] / df['Num_Words']
    df["count_words_title"] = df['TRANSACTION DETAILS'].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
    df["mean_word_len"] = df['TRANSACTION DETAILS'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
    df['punct_percent']= df['num_punctuation']*100 / df['Num_Words']

In [65]:
CV_ = CountVectorizer(min_df = 4, max_features = 5000, ngram_range = (3, 7))
CV_.fit(train['TRANSACTION DETAILS'])

train_words = pd.DataFrame(CV_.transform(train['TRANSACTION DETAILS']).toarray())
test_words = pd.DataFrame(CV_.transform(test['TRANSACTION DETAILS']).toarray())

In [50]:
LDA = LatentDirichletAllocation(n_components = 20, max_iter = 100, random_state = random_seed)
LDA.fit(pd.concat([train_words,test_words]))
Topics = [f'Topic_{x}' for x in range(0, 20)]
train[Topics] = LDA.transform(train_words)
test[Topics] = LDA.transform(test_words)

In [51]:
# n_top_words = 24
# topic_summaries = []

# # get topics and topic terms
# topic_word = LDA.components_ 
# vocab = CV_.get_feature_names()

# for i, topic_dist in enumerate(topic_word):
#     topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
#     topic_summaries.append(' '.join(topic_words))
#     print('Topic {}: {}'.format(i, ' | '.join(topic_words)))

In [52]:
#Duplicated
cols = train.columns
dup = []
for feat_1 in cols:
    if (feat_1 in dup):
        continue
    for feat_2 in cols.drop(feat_1):
        if (feat_2 in dup):
            continue
        if (train[feat_1].equals(train[feat_2])):
            train.drop(feat_2,inplace=True,axis=1)
            test.drop(feat_2,inplace=True,axis=1)
            dup.append(feat_2)

In [53]:
#Constant
for feat in test.columns:
    if ((len(train[feat].value_counts().keys()) == 1) | (len(test[feat].value_counts().keys()) == 1)):
        train.drop(feat,inplace=True,axis=1)
        test.drop(feat,inplace=True,axis=1)

In [54]:
# separating data into train and test
# train = data[data.Category.notnull()].reset_index(drop = True)
# test = data[data.Category.isna()].reset_index(drop = True)
# test.drop('Category', axis = 1, inplace = True)
print('shape', train.shape, test.shape)

shape (56203, 303) (22625, 302)


In [55]:
le = LabelEncoder()
features = ['TRANSACTION DETAILS']
full_data = pd.concat([train[features], test[features]], axis = 0)
for col in (features):
    le.fit(full_data[col].values)
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])
    print(col)

TRANSACTION DETAILS


In [56]:
X, y = train.drop('Category', axis = 1), train['Category']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 121)

In [57]:
# make sure validation set has all the categories as training set
print('validation set is missing cols {}'.format(set(y_train.unique()) - set(y_val.unique())))
missing_cols = list(set(y_train.unique()) - set(y_val.unique()))

validation set is missing cols {'Entertainment', 'Donations'}


In [58]:
# X_sam, y_sam = RandomOverSampler(random_state = 42).fit_resample(X, y)
# skf = StratifiedKFold(n_splits = 5, random_state = random_seed, shuffle = True)
# X_Train, X_Val, y_Train, y_Val = train_test_split(X_sam, y_sam, test_size = 0.3, random_state = 121)

In [59]:
# cat_scores = []
# for fold,(tr_in,te_in) in enumerate(skf.split(X, y)):
#     print("===== Fold {fold} =====".format(fold = fold + 1))
#     X_train, X_test = X.iloc[tr_in], X.iloc[te_in]
#     y_train, y_test = y.iloc[tr_in], y.iloc[te_in]
#     cat = CatBoostClassifier(iterations = 1300, learning_rate = 0.09, depth = 6, reg_lambda = 17.56889442852513, verbose = 0, random_state = random_seed)
#     cat.fit(X_train, y_train, eval_set = [(X_train, y_train),(X_test, y_test)], early_stopping_rounds = 500, use_best_model = True)
#     loss = log_loss(y_test, cat.predict_proba(X_test))
#     cat_scores.append(loss)
#     print(loss)

# print('cat_score : ' + str(np.mean(cat_scores)))

In [68]:
rf = RandomForestClassifier(n_estimators = 600, max_depth = 10, random_state = 2001)

# Training the model on the training dataset
rf.fit(X_train, y_train)

y_pred = rf.predict(X_val)
y_val_dummies = pd.get_dummies(pd.concat([y_val, pd.Series([col for col in missing_cols])], ignore_index=True)).iloc[:-len(missing_cols)][rf.classes_]
y_pred_poba = rf.predict_proba(X_val)

# Calculate accuracy
print("ACCURACY OF THE MODEL: ", accuracy_score(y_val, y_pred), 'The log loss of the model:',log_loss(y_val_dummies, y_pred_poba))

# Classification report
print(classification_report(y_val, y_pred))

NameError: name 'X_train' is not defined

In [ ]:
cat = CatBoostClassifier(l2_leaf_reg = 9.861413522475084, depth = 10, bootstrap_type = 'Bayesian', learning_rate = 0.02146, n_estimators = 3167,
                         leaf_estimation_iterations = 1, random_strength = 0.18295032711212016, loss_function = 'MultiClass', verbose = 0, random_state = random_seed)

# Training the model on the training dataset
cat.fit(X_train, y_train)

y_pred = cat.predict(X_val)
y_val_dummies = pd.get_dummies(pd.concat([y_val, pd.Series([col for col in missing_cols])], ignore_index=True)).iloc[:-len(missing_cols)][cat.classes_]
y_pred_poba = cat.predict_proba(X_val)

# Calculate accuracy
print("ACCURACY OF THE MODEL: ", accuracy_score(y_val, y_pred), 'The log loss of the model:',log_loss(y_val_dummies, y_pred_poba))

# Classification report
print(classification_report(y_val, y_pred))

In [66]:
pred = rf.predict_proba(test)
 # * 0.4 + cat.predict_proba(test) * 0.6
predictions = pd.DataFrame(pred, columns = rf.classes_, index = sub.ID)
predictions.drop('Entertainment', axis = 1, inplace = True)

NameError: name 'rf' is not defined

In [67]:
predictions.to_csv('first_2.csv', index = True)

NameError: name 'predictions' is not defined